In [1]:
from qiskit import QuantumCircuit, transpile
import numpy as np
import qiskit.quantum_info as qi
from scipy.optimize import minimize
import random

def sample_floats(low, high, k = 1):
    result = []
    seen = set()
    for i in range(k):
        x = random.uniform(low, high)
        while x in seen:
            x = random.uniform(low, high)
        seen.add(x)
        result.append(x)
    return result


qc_x = QuantumCircuit(1)
U_param = sample_floats(-np.pi, np.pi, 3)
print(list(np.around(np.array(U_param),3)))
qc_x.u(U_param[0], U_param[1], U_param[2], 0)
#qc_x = transpile(qc_x, basis_gates=['rx', 'rz', 'p', 'x'])
print(qc_x)
op = qi.Operator(qc_x)
array = op.data
print(array)

[-2.604, -3.135, -2.493]
   ┌───────────────────────────┐
q: ┤ U(-2.6044,-3.1347,-2.493) ├
   └───────────────────────────┘
[[ 0.26538177+0.j         -0.76833414-0.58243898j]
 [ 0.96412082+0.00659961j  0.21038267+0.16176098j]]


In [3]:
def x_param(param, out = False):
    qc_param = QuantumCircuit(1)
    qc_param.rx(param[0], 0)
    qc_param.rz(param[1], 0)
    qc_param.rx(param[2], 0)
    qc_param.rz(param[3], 0)
    qc_param.rx(param[4], 0)
    qc_param.rz(param[5], 0)
    
    qc_param.p(param[6], 0)
    qc_param.x(0)
    qc_param.p(param[6], 0)
    qc_param.x(0)
    
    op_param = qi.Operator(qc_param)
    array_param = op_param.data
    if(out):
        print(qc_param)
        print(array_param)
    return array_param

def loss(param):
    norm_ = np.linalg.norm(np.array(array) - np.array(x_param(param)), ord = 'fro')
    return norm_

In [4]:
bounds = [[-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi], [-np.pi, np.pi]]
param0 = [0, 0, 0, 0, 0, 0, 0]
res = minimize(loss, param0, method='Nelder-Mead',#'powell'
               options={'xatol': 1e-8, 'disp': True}, bounds=bounds)# 

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 609
         Function evaluations: 968


In [5]:
print(list(np.around(np.array(res.x),3)))
x_param(res.x, True)
loss(res.x)

[0.799, 0.192, 0.739, -0.61, 1.351, 2.672, 0.328]
   ┌─────────────┐┌─────────────┐┌─────────────┐┌──────────────┐┌───────────┐»
q: ┤ Rx(0.79892) ├┤ Rz(0.19195) ├┤ Rx(0.73903) ├┤ Rz(-0.60978) ├┤ Rx(1.351) ├»
   └─────────────┘└─────────────┘└─────────────┘└──────────────┘└───────────┘»
«   ┌───────────┐┌────────────┐┌───┐┌────────────┐┌───┐
«q: ┤ Rz(2.672) ├┤ P(0.32774) ├┤ X ├┤ P(0.32774) ├┤ X ├
«   └───────────┘└────────────┘└───┘└────────────┘└───┘
[[ 0.26538177+9.11174414e-10j -0.76833414-5.82438979e-01j]
 [ 0.96412082+6.59960792e-03j  0.21038267+1.61760982e-01j]]


2.8234566506819784e-09